markdown docs.

In [1]:
import IPython

import ipykernel.ipkernel
from ipython_genutils.py3compat import builtin_mod, PY3, unicode_type, safe_unicode

from IPython.utils.tokenutil import token_at_cursor, line_at_cursor


try:
    import stringdisplays, markdown, tokens, phrases
except:
    with __import__('importnb').Notebook(): from pidgin import stringdisplays, markdown, tokens, phrases

import sys
import textwrap
import black
import mimetypes
import collections, html
import base64
import sys
import IPython
from traitlets import *
ip = self = IPython.get_ipython()
import trio

In [8]:
class PidginShell(ipykernel.zmqshell.ZMQInteractiveShell):
    def user_expressions(self, expressions, show=True): 
        return trio.run(self.async_user_expressions, expressions, show)

    async def async_user_expressions(self, expressions, show=True):
        async with trio.open_nursery() as nursery: [
            nursery.start_soon(self.single_user_expression, expression, show)
            for key, expression in expressions.items()]
        return {}
        
    async def single_user_expression(self, code, show=True, **expressions):
        for expression in map(str.strip, code.strip('`').split(';')):
            key = f"`{expression}`"
            expressions[key] = self.input_transformer_manager.transform_cell(expression)
        for key, expression in super().user_expressions(expressions).items(): 
            data = self.show_expression(key, expression, show)
        return data
    
    def show_expression(self, code, data, show=True):
        if data['status'] != 'ok': data['data'] = {'text/plain': '\n'.join(data['data']['traceback'])}
        self.auto_rewrite_input(code.strip('`')), sys.stdout.flush(), sys.stderr.flush()
        if data['status'] == 'ok':
            display_id = self.user_ns.get(code, IPython.display.DisplayHandle())
            if show:  setattr(__import__('__main__'), code, display_id)
            if code in self.user_ns: IPython.display.publish_display_data(data['data'], transient={'display_id': display_id.display_id}, update=True)
            IPython.display.publish_display_data(data['data'], transient={'display_id': display_id and display_id.display_id})
        elif data['status'] == 'error':
            list(map(sys.stderr.write, data['traceback']))
            sys.stderr.flush()
        return data

    def update_expression(self, code, data, show=True):
        if data['status'] != 'ok': data['data'] = {'text/plain': '\n'.join(data['data']['traceback'])}
        self.auto_rewrite_input(code.strip('`')), sys.stdout.flush(), sys.stderr.flush()
        if data['status'] == 'ok':
            display_id = self.user_ns.get(code, IPython.display.DisplayHandle())
            if show:  setattr(__import__('__main__'), code, display_id)
            if code in self.user_ns: IPython.display.publish_display_data(data['data'], transient={'display_id': display_id.display_id}, update=True)
            IPython.display.publish_display_data(data['data'], transient={'display_id': display_id and display_id.display_id})
        elif data['status'] == 'error':
            list(map(sys.stderr.write, data['traceback']))
            sys.stderr.flush()
        return data
    
    def object_inspect_mime(self, code, cursor_pos=None, detail_level=0):
        token, start, end = tokens.tokenize(code, cursor_pos, *(x for x in dir(__import__('__main__')) if ' ' in x and not(x[0]==x[-1]=='`')))
        if token.endswith('??'): detail_level=1
        inline = token and (token[0] == token[-1] == '`')
        if inline: 
            data = trio.run(self.single_user_expression, token)['data']
        else:
            try:
                data = super(type(self), self).object_inspect_mime(
                    token_at_cursor(token, len(token)), detail_level=detail_level
                )
            except KeyError: data = {'text/markdown': code}
        if not detail_level:
            if not inline:
                object = self.user_ns.get(token, sys.modules.get(token, None))
                if object is not None: data.update(self.display_formatter.format(object)[0])
        return data
    

Each uesr expression is evaluated in serial to provide the correct output.  The output is from the last expressioon.

In [10]:
import trio

In [11]:
    import ipykernel

In [12]:
import trio

In [13]:
    import importlib

In [4]:
def requote(str, token='"""'):
    if token in str: token = "'''"
    return token+str+token

    ipykernel.ipkernel.IPythonKernel.do_execute??

In [5]:
ipykernel.ipkernel.IPythonKernel.help_links.default_args = tuple([
    ipykernel.ipkernel.IPythonKernel.help_links.default_args[0] + [
        {'text': 'Markdown Cheat Sheet', 'url': 'https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet'},
        {'text': 'GraphViz DOT Language', 'url': 'https://www.graphviz.org/doc/info/lang.html'}
    ]
] + list(ipykernel.ipkernel.IPythonKernel.help_links.default_args[1:]))

In [6]:
class PidginKernel(ipykernel.ipkernel.IPythonKernel):
    shell_class = Type(PidginShell)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.pidgin_history = collections.OrderedDict()
        stringdisplays.load_ipython_extension(self.shell)
        phrases.load_ipython_extension(self.shell)
        self.shell.user_ns['__import__'] = importlib.import_module
        
    language_info = {**ipykernel.ipkernel.IPythonKernel.language_info, **{
        'name': 'pidgin',
        'display_name': 'pidgin',
        'nbconvert_exporter': 'python',
        'file_extension': '.md.ipynb',
        "argv": "python -m pidgin.kernel -f {connection_file}".split(),
    }}        
    
    def update_code(self, code, display_id=None):
        if code.strip() and code.splitlines()[0].strip(): self.display_id = IPython.display.display(
                IPython.display.Markdown(code), display_id=display_id or self.display_id).display_id
            
    def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
        if user_expressions is None: user_expressions = {}
        #define_urls(code)
        if not code.lstrip().startswith('%'): source = markdown.renderer(code, user_expressions=user_expressions)
        else: source = code
        silent = silent or not source
        if source: self.update_code(code, True)
            
        if not source and not user_expressions:
            result = super().do_execute(requote(code), False, store_history=False, user_expressions={}, allow_stdin=allow_stdin)
        else: result = super().do_execute(source, silent, store_history=store_history, user_expressions=user_expressions, allow_stdin=allow_stdin)
        return result
    
    def do_inspect(self, code, cursor_pos, detail_level=0):
        data = self.shell.object_inspect_mime(code, cursor_pos, detail_level=detail_level,)
        return {'status': 'ok', 'metadata': {}, 'found': bool(data), 'data': data,}

In [7]:
import re

In [8]:
def define_urls(str):
    while str[0] != 'h' and str[0] == str[1]: str = str[1:-1]
    for body in str.split('http')[1:]:
        setattr(__import__('__main__'), f"http{body.split()[0]}", f"http{body.split()[0]}" )

In [9]:
import jupyter_client.manager

In [10]:
manager = jupyter_client.kernelspec.KernelSpecManager()
import json

In [11]:
import pidgin

In [12]:
import json

    %%file kernel.py
    with __import__('importnb').Notebook():
        from .kernel import PidginKernel
        
    if __name__ == '__main__':
        from ipykernel.kernelapp import IPKernelApp
        IPKernelApp.launch_instance(kernel_class=PidginKernel)
    

In [13]:
if __name__ == '__main__':

    with open('C:/Users/deathbeds/pidgin/pidgin/kernel.json', 'w') as f:
        json.dump(PidginKernel.language_info, f)
    manager.install_kernel_spec('C:/Users/deathbeds/pidgin/pidgin/')
    manager.get_all_specs()        